# 1-Generate Predictions using LangChain

- **Goal:** Prediction Recognition

- **Purpose:** To implement step 1 with sub steps of prediction recognition pipeline. See steps
    1. Generate predictions
        1. Create several prediction prompts templates
        2. Utilize open-source LLMs to generate predictions

- **Misc:**
    - `%store`: Cell magic will store the variable of interest so we can load in another notebook

In [1]:
import os, sys

import pandas as pd

from tqdm import tqdm
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from log_files import LogData
from data_processing import DataProcessing
from text_generation_models import TextGenerationModelFactory

In [2]:
# pd.set_option('max_colwidth', 800)

groq_cloud_tgmf = TextGenerationModelFactory()
llama_versatile_generation_model = groq_cloud_tgmf.create_instance(model_name='llama-3.3-70b-versatile')
llama_instant_generation_model = groq_cloud_tgmf.create_instance('llama-3.1-8b-instant')
llama_70b_8192_generation_model = groq_cloud_tgmf.create_instance('llama3-70b-8192')
llama_8b_8192_generation_model = groq_cloud_tgmf.create_instance('llama3-8b-8192')

navi_gator_tgmf = TextGenerationModelFactory()
gpt_35_turbo_generation_model = navi_gator_tgmf.create_instance('gpt-3.5-turbo')
gpt_4_turbo_generation_model = navi_gator_tgmf.create_instance('gpt-4o')
mixtral_87b_instruct_generation_model = navi_gator_tgmf.create_instance('mixtral-8x7b-instruct') 

## LangChain Templates for Domain Predictions

In [3]:
full_prediction_template = """{prediction_properties}

{prediction_requirements}

{prediction_templates}

{prediction_examples}
"""

full_prediction_prompt = PromptTemplate.from_template(full_prediction_template)

In [4]:
prediction_properties_template = """A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_a>), where it consists of the following four properties:

    1. <p_s>, any source entity in the {prediction_domain} domain.
        - Can be a person (with a name) or a {prediction_domain} person such as a {prediction_domain} reporter, {prediction_domain} analyst, {prediction_domain} expert, {prediction_domain} top executive, {prediction_domain} senior level person, etc).
        - Can only be an organization that is associated with the {prediction_domain} prediction.
    2. <p_t>, any target entity in the {prediction_domain} domain.
	    - Can be a person (with a name) or a {prediction_domain} person such as a {prediction_domain} reporter, {prediction_domain} analyst, {prediction_domain} expert, {prediction_domain} top executive, {prediction_domain} senior level person, etc).
        - Can only be an organization that is associated with the {prediction_domain} prediction.
    3. <p_d>, date range when <p> is expected to come to fruition.
        - Forecast can range from a second to anytime in the future.
        - Answers the questions: "How far to go out from today?" or "Where to stop?".
    4. <p_a>, {prediction_domain} prediction attribute.
        - Characteristics of a domain-specific attributes such as various quantifiable metrics relevant to the {prediction_domain} domain.
        - Some examples are {prediction_domain_attribute}.  
"""
prediction_properties_prompt = PromptTemplate.from_template(prediction_properties_template)

In [5]:
prediction_requirements_template = """{prediction_domain} requirements to use for each prediction:

    - Should be based on real-world {prediction_domain} data and not hallucinate.
    - Only a simple sentence (prediction) (and NOT compounding using "and" or "or").
    - Should diversify all four properties of the prediction (<p>) as in change and not use same for <p_s>, <p_t>, <p_d>, <p_a>.
    - Should use synonyms to predict such as forecasts, speculates, foresee, envision,according etc., and not use any of them more than ten times.
    - The prediction should be unique and not repeated.
    - Do not number the predictions.
    - Do not say, "As the {prediction_domain}, I will generate company-based {prediction_domain} predictions using the provided templates." or anything similar.
    - Use the five different templates and examples provided.
    - Change how the current date (<p_d>) written in the prediction with examples of (1) Wednesday, August 21, 2024; (2) Wed, August 21, 2024; (3) 08/21/2024; (4) 08/21/2024; (5) 21/08/2024; (6) 21 August 2024; (7) 2024/08/21; (8) 2024-08-21; (9) August 21, 2024; (10) Aug 21, 2024; (11) 21 August 2024, (12) 21 Aug 2024, Q3 of 2027, 2029 of Q3, etc (with removing day of week).
    {domain_requirements}
    - Do not say, "Here are {predictions_N} unique {prediction_domain} predictions based on the provided templates and examples:" in the prompt.
    - Do not use any of the examples in the prompt.
    - In front of every prediction, put the template number in the format of "T1:", "T2:", etc. and do not number them like "1.", "2.", etc.
    - Do not put template number on line by itself. Always pair with a prediction.
    - Disregard brackets: "<>"
    - Should never say "Here are {predictions_N} unique {prediction_domain} predictions based on the provided templates and examples:" or "Note: I've made sure to follow the guidelines and templates provided, and generated unique predictions that meet the requirements."
    - Do not use person name of entity name more than once as in don't use name Joe as both the <p_s> and <p_t>, unless like Mr. Sach and Goldman Sach or Mr. Sam Walton and Sam's Club, etc.
    - Should variate the slope of rise/increase/as much as, fall/decrease/as little as, change, stay stable, high/low chance/probability/degree of, etc.
    - Should variate the prediction verbs such as will, would, be going to, should, etc.
"""
prediction_requirements_prompt = PromptTemplate.from_template(prediction_requirements_template)

In [6]:
prediction_templates_template = """Here are some {prediction_domain} templates:

- {prediction_domain} template 1: <p_s> forecasts that the <p_a> at <p_t> will likely decrease in <p_d>.
- {prediction_domain} template 2: On <p_d>, <p_s> speculates that the <p_a> at <p_t> will likely increase.
- {prediction_domain} template 3: <p_s> predicts that on <p_d>, the <p_t> <p_a> will likely rise.
- {prediction_domain} template 4: According to <p_s>, the <p_a> at <p_t> will likely fall in <p_d>.
- {prediction_domain} template 5: In <p_d>, <p_s> envisions that <p_t> <p_a> will likely stay stable.
- {prediction_domain} template 6: <p_t> <p_a> will likely rise in <p_d>, according to <p_s>. 
"""
prediction_templates_prompt = PromptTemplate.from_template(prediction_templates_template)

In [7]:
prediction_examples_template = """Here are some examples of {prediction_domain} predictions:

{domain_examples}

With the above, generate a unique set of {predictions_N} predictions. Think from the perspective of an {prediction_domain} analyst, expert, top executive, or senior level person."""
prediction_examples_prompt = PromptTemplate.from_template(prediction_examples_template)

In [8]:
prediction_input_prompts = [
    ("prediction_properties", prediction_properties_prompt),
    ("prediction_requirements", prediction_requirements_prompt),
    ("prediction_templates", prediction_templates_prompt),
    ("prediction_examples", prediction_examples_prompt),
]

pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prediction_prompt, pipeline_prompts=prediction_input_prompts
)

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_75446/3566428802.py:8: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  pipeline_prompt = PipelinePromptTemplate(


## Generate Domain Predictions

In [9]:
predictions_N = 2

### Generate Financial Predictions

In [10]:
financial_attributes = """stock price, envision, net profit, revenue, operating cash flow, research and development expenses, operating income, gross profit."""
financial_requirements = """- Should be based on real-world financial earnings reports.
    - Suppose the time when $p$ was made is during any earning season.
    - Include stocks from all sectors such as consumer staples, energy, finance, health care, industrials, materials, media, real estate, retail, technology, utilities, defense, etc.
    - Include the US Dollar sign ($) before or USD after the amount of the financial attribute."""

financial_examples = """
 - financial examples for template 1:
    1. Detravious, an investor forecasts that the stock price at Apple will likely decrease in 2025 Q1.
    2. Ava Lee predicts that the operating cash flow at ExxonMobil should decrease in 03/21/2025 to 08/21/2025.
    3. Detravious, an investor predicts that the stocks he has will likely increase in 2024/08/21.
 - financial examples for template 2:
    1. On March 15, 2025, Goldman Sachs speculates that the interest rates at the Federal Reserve will likely increase.
    2. On April 2, 2025, Morgan Stanley forecasts that the stock value at Tesla will likely increase.
    3. On January 28, 2025, Chase analysts foresee that there stock prices will likely increase.
 - financial examples for template 3:
    1. Morgan Stanley anticipates that on May 3, 2025, the NASDAQ composite index could climb moderately.
    2. BlackRock foresees that on April 22, 2025, the value of Bitcoin has a high probability of rising sharply.
    3. Morgan Stanley predicts that on May 3, 2025, there stock price will likely rise.
 - financial examples for template 4:
    1. According to Chase Bank, the expected returns at emerging market equities will likely fall in May 2025.
    2. According to Ryan, the projected revenue at Meta Platforms will likely fall in Q2 2025.
    3. According to Apple, the trading volume it has will likely increase in Q1 2025
 - financial examples for template 5:
    1. In April 2025, Wells Fargo expects that U.S. Treasury yields will likely stay stable.
    2. In May 2025, Bob envisions that the inflation rate in Wells Fargo will likely stay stable.
    3. In August 2025, Bob predicts that the stocks he has will likely stay stable.
 - financial examples for template 6:
    1. Apple stock price will decrease in February 2025, accordeing to Roger.
    2. The NASDAQ index is expected to rise in June 2025, according to Bank of America.
    3. Roger foresees the stock price increasing in July 2025, according to his projections.
 """

In [11]:
financial_input_dict = {
    "prediction_domain": "financial",
    "prediction_domain_attribute": financial_attributes,
    "domain_requirements": financial_requirements,
    "domain_examples": financial_examples,
    "predictions_N": predictions_N
}
financial_prompt_output = pipeline_prompt.format(**financial_input_dict)
print(financial_prompt_output)


A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_a>), where it consists of the following four properties:

    1. <p_s>, any source entity in the financial domain.
        - Can be a person (with a name) or a financial person such as a financial reporter, financial analyst, financial expert, financial top executive, financial senior level person, etc).
        - Can only be an organization that is associated with the financial prediction.
    2. <p_t>, any target entity in the financial domain.
	    - Can be a person (with a name) or a financial person such as a financial reporter, financial analyst, financial expert, financial top executive, financial senior level person, etc).
        - Can only be an organization that is associated with the financial prediction.
    3. <p_d>, date range when <p> is expected to come to fruition.
        - Forecast can range from a second to anytime in the future.
        - Answers the questions: "How far to go out from today?" or "Where to stop?".
    4. 

### Generate Health Predictions

In [12]:
health_attributes = """obesity rates, prevalence of chronic illnesses, average physical activity levels, nutritional intake, etc."""
health_requirements = """- Should be based on real-world health reports.
    - Suppose the time when $p$ was made is during any season such as flu season, allergy season, pandemic, epidemic, etc.
    - Include reports from all Health organization, researcher, doctor, physical therapist, physician assistant, nurse practictioners, fitness expert, etc."""

# For each template, have a rise, fall, or stable example, respectively.?
health_examples = """
- health examples for template 1:
    1. CDC predicts that the obesity rates at the national level will likely decrease in late 2025.
    2. CDC forecasts that the prevalence of chronic illnesses at urban health centers will likely decrease in Q3 2025.
    3. Chase Thinks that the average physical activity levels he does will likely increase in 2025.
- health examples for template 2:
    1. On May 15, 2025, the CDC speculates that the average physical activity levels at U.S. high schools will likely increase.
    2. On June 1, 2025, Sam speculates that the nutritional intake at rural clinics in America will likely increase.
    3. On July 1, 2025, Sam predicts that the obesity rates at the national level will likely increase.
- health examples for template 3:
    1. The NIH predicts that on July 5, 2025, public engagement in preventative health screenings will likely rise.
    2. Alex suspects that on June 15, 2025, the obesity rates at the national level will likely decrease.
    3. Talon envisions that on Januaruy 3, 2025, the average physical activity levels for him will likely rise.
- health examples for template 4:
    1. According to the CDC, the obesity rates at U.S. elementary schools will likely fall in Fall 2025.
    2. According to the NIH, the average sugar consumption at public school cafeterias will likely fall in September 2025.
    3. According to James, the average physical activity levels for him will likely fall in 2025.
- health examples for template 5:
    1. In June 2025, Dr. Maria Thompson envisions that national obesity rates will likely decrease.
    2. In August 2025, Professor James Liu envisions that average physical activity levels among teenagers will likely increase.
    3. In July 2025, Dr. Aisha Reynolds envisions that her nutritional intake will likely stay stable.
- health examples for template 6:
    1. Physical activity levels among seniors will likely rise in July 2025, according to Dr. Elena Morales.
    2. Nutritional awareness in public schools will likely rise in September 2025, according to Professor Daniel Kim.
    3. Sarah's health screening participation will likely rise in late 2025, according to Sarah.

Want these tied to specific campaigns, regions, or health studies?
"""

In [13]:
health_input_dict = {
    "prediction_domain": "health",
    "prediction_domain_attribute": health_attributes,
    "domain_requirements": health_requirements,
    "domain_examples": health_examples,
    "predictions_N": predictions_N
}

health_prompt_output = pipeline_prompt.format(**health_input_dict)


### Generate Policy Predictions

In [14]:
policy_attributes = """election outcomes, economic reforms, legislative impacts."""
policy_requirements = """- Should be based on real-world policy reports.
    - Suppose the time when $p$ was made is during an election cycle or non-election cycles.
    - Include policies & laws, from all sectors such as consumer staples, energy, finance, health care, industrials, materials, media, real estate, retail, technology, utilities, defense, etc."""

policy_examples = """
 - policy examples for template 1:
    1. Election outcomes in key swing states will likely rise in national importance in November 2025, according to analyst Rachel Lin.
    2. Economic reforms in the energy sector will likely increase in visibility in Q3 2025, according to Dr. Marcus Grant from the Brookings Institution.
    3. Angela Cortez forcasts that data privacy laws in the technology sector will likely stay stable in the public, in late 2025, according to expert Angela Cortez.
 - policy examples for template 2:
    1. On April 5, 2025, the Brookings Institution speculates that voter turnout at battleground states will likely stay stable.
    2. On May 12, 2025, the International Monetary Fund speculates that investment activity at emerging markets will likely increase.
    3. On June 1, 2025, policy analyst Rachel Kim speculates that regulatory scrutiny in Rachels large tech firms will likely decrease.
 - policy examples for template 3:
    1. Representative Angela Brooks predicts that on October 15, 2025, the defense budget allocation will likely stay stable.
    2. Economist Dr. Henry Zhao predicts that on July 1, 2025, the corporate tax rate in the finance sector will likely rise.
    3. Senator Michael Greene predicts that on November 3, 2025, his voter engagement in suburban districts will likely decrease.
 - policy examples for template 4:
    1. According to Senator Alicia Ramirez, the public trust at federal institutions will likely increase in late 2025.
    2. According to Thomas Nguyen, the investment confidence at the real estate sector will likely fall in Q3 2025.
    3. According to policy advisor Natalie Chen, the employment rate at at her company firms will likely stay stable in September 2025.
 - policy examples for template 5:
    1. In August 2025, Senator Jordan Ellis envisions that healthcare spending will likely stay stable.
    2. In June 2025, economist Dr. Priya Nandakumar envisions that inflation rates in the consumer staples sector will likely increase.
    3. In October 2025, policy strategist Kevin Adler envisions that his defense contract approvals will likely decrease.
 - policy examples for template 6:
    1. Renewable energy investments are expected to rise in Q3 2025, according to Dr. Elena Foster.
    2. Healthcare subsidies will likely decrease in September 2025, according to Senator Marcus Lee.
    3. Rachel Alvarez's voter participation in local elections will likely stay the same in November 2025, according to Rachel Alvarez.
"""

In [15]:
policy_input_dict = {
    "prediction_domain": "policy",
    "prediction_domain_attribute": policy_attributes,
    "domain_requirements": policy_requirements,
    "domain_examples": policy_examples,
    "predictions_N": predictions_N
}

### Generate Weather Predictions

In [16]:
weather_attributes = """temperature, precipitation, wind speed, humidity, etc."""
weather_requirements = """- Should be based on real-world weather reports.
    - Suppose the time when $p$ was made is during any season and any location (ie: Florida known for hurricanes, California known for wildfires, etc).
    - Include reports from all meteorologists, weather organizations, or any type of weather entity.."""

# For each template, have a rise, fall, or stable example, respectively.?
weather_examples = """
- weather examples for template 1:
    1. The National Weather Service forecasts that the precipitation levels at Miami will likely increase in September 2025.
    2. AccuWeather forecasts that the humidity at Phoenix will likely stay stable in early fall 2025.
    3. Sam forecasts that the wind speed at his house will likely decrease in November 2025.
- weather examples for template 2:
    1. On August 10, 2025, Meteorologist Lisa Park speculates that the temperature at Los Angeles will likely increase.
    2. On June 15, 2025, Dr. Mark Williams speculates that the humidity at Houston will likely decrease.
    3. On July 5, 2025, San Francisco's meteorological team speculates that the wind speed in San Francisco will likely stay stable.
- weather examples for template 3:
    1. Dr. Anna Lee predicts that on May 20, 2025, the temperature at Denver will likely decrease.
    2. Meteorologist John Roberts predicts that on July 1, 2025, the wind speed at New York will likely rise.
    3. The Miami Weather Bureau predicts that on August 10, 2025, the humidity at Miami will likely stay stable.
- weather examples for template 4:
    1. According to Dr. Linda Garcia, the temperature at Boston will likely increase in November 2025.
    2. According to Meteorologist Jake Wilson, the precipitation levels at Seattle will likely stay stable in January 2025.
    3. According to Dylan, the wind speed at Dylans home will likely fall in October 2025.
- weather examples for template 5:
    1. In December 2025, Meteorologist Claire Thompson envisions that the temperature at Chicago will likely stay increase.
    2. In May 2025, Dr. Robert Harris envisions that the humidity at Phoenix will likely stay stable.
    3. In July 2025, the San Francisco Weather Bureau envisions that the precipitation levels in San Francisco will likely decrease.
- weather examples for template 6:
    1. Temperature in Las Vegas will likely rise in July 2025, according to Meteorologist Nina Patel.
    2. Humidity in Houston will likely decrease in August 2025, according to Dr. Kevin Morales.
    3. Wind speed in Miami will likely stay stable in October 2025, according to the Miami Weather Bureau.
"""

In [17]:

weather_input_dict = {
    "prediction_domain": "weather",
    "prediction_domain_attribute": weather_attributes,
    "domain_requirements": weather_requirements,
    "domain_examples": weather_examples,
    "predictions_N": predictions_N
}

In [18]:
N_batches = 1
# text_generation_models = [llama_instant_generation_model]
# text_generation_models = [llama_versatile_generation_model]
text_generation_models = [llama_versatile_generation_model, llama_instant_generation_model, llama_70b_8192_generation_model, 
                          llama_8b_8192_generation_model, gpt_35_turbo_generation_model, gpt_4_turbo_generation_model, 
                          mixtral_87b_instruct_generation_model]

# text_generation_models = [llama_versatile_generation_model, llama_instant_generation_model, llama_70b_8192_generation_model, 
#                           llama_8b_8192_generation_model, gpt_35_turbo_generation_model]

In [20]:
prediction_domains = ["finance", "health", "policy", "weather"]
prediction_prompt_outputs = {
    "finance": financial_prompt_output,
    "health": health_prompt_output,
    "policy": financial_prompt_output,
    "weather": financial_prompt_output,
}
prediction_label = 1

batched_predictions_df = groq_cloud_tgmf.batch_generate_data(N_batches=N_batches, 
                                text_generation_models=text_generation_models, 
                                domains=prediction_domains,
                                prompt_outputs=prediction_prompt_outputs,
                                sentence_label=prediction_label)

  0%|          | 0/1 [00:00<?, ?it/s]

===================================== Batch 0 ===============================================
finance --- llama-3.3-70b-versatile --- GROQ_CLOUD
finance --- llama-3.1-8b-instant --- GROQ_CLOUD
finance --- llama3-70b-8192 --- GROQ_CLOUD
finance --- llama3-8b-8192 --- GROQ_CLOUD
finance --- gpt-3.5-turbo --- NAVI_GATOR
finance --- gpt-4-turbo --- NAVI_GATOR
finance --- mixtral-8x7b-instruct --- NAVI_GATOR

health --- llama-3.3-70b-versatile --- GROQ_CLOUD
health --- llama-3.1-8b-instant --- GROQ_CLOUD
health --- llama3-70b-8192 --- GROQ_CLOUD
health --- llama3-8b-8192 --- GROQ_CLOUD
health --- gpt-3.5-turbo --- NAVI_GATOR
health --- gpt-4-turbo --- NAVI_GATOR
health --- mixtral-8x7b-instruct --- NAVI_GATOR

policy --- llama-3.3-70b-versatile --- GROQ_CLOUD
policy --- llama-3.1-8b-instant --- GROQ_CLOUD
policy --- llama3-70b-8192 --- GROQ_CLOUD
policy --- llama3-8b-8192 --- GROQ_CLOUD
policy --- gpt-3.5-turbo --- NAVI_GATOR
policy --- gpt-4-turbo --- NAVI_GATOR
policy --- mixtral-8x7b-ins

100%|██████████| 1/1 [00:25<00:00, 25.33s/it]


Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/data/prediction_logs
Save CSV: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/data/prediction_logs/batch_2-prediction/batch_2-from_df.csv

CSV to Log


In [21]:
batched_predictions_df

[                                       Base Sentence  Sentence Label   Domain  \
 0  T1: JPMorgan Chase forecasts that the revenue ...               1  finance   
 1  T2: On 2024-08-21, Citigroup speculates that t...               1  finance   
 
                 Model Name    API Name  
 0  llama-3.3-70b-versatile  GROQ_CLOUD  
 1  llama-3.3-70b-versatile  GROQ_CLOUD  ,
                                        Base Sentence  Sentence Label   Domain  \
 0  T1: Morgan Stanley forecasts that the stock pr...               1  finance   
 1  T2: On August 21, 2024, Goldman Sachs speculat...               1  finance   
 2  T3: Morgan Stanley predicts that on September ...               1  finance   
 3  T4: According to BlackRock, the expected retur...               1  finance   
 4  T5: In Q1 2028, Wells Fargo expects that U.S. ...               1  finance   
 5  T6: Apple stock price will likely rise in 2029...               1  finance   
 
              Model Name    API Name  
 0  llama-

In [22]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
batched_predictions_df

[                                                                                    Base Sentence  \
 0     T1: JPMorgan Chase forecasts that the revenue at Amazon will likely decrease in Q3 of 2027.   
 1  T2: On 2024-08-21, Citigroup speculates that the net profit at Microsoft will likely increase.   
 
    Sentence Label   Domain               Model Name    API Name  
 0               1  finance  llama-3.3-70b-versatile  GROQ_CLOUD  
 1               1  finance  llama-3.3-70b-versatile  GROQ_CLOUD  ,
                                                                                                       Base Sentence  \
 0                      T1: Morgan Stanley forecasts that the stock price at Amazon will likely decrease in 2027 Q2.   
 1  T2: On August 21, 2024, Goldman Sachs speculates that the operating cash flow at Microsoft will likely increase.   
 2                 T3: Morgan Stanley predicts that on September 15, 2025, the S&P 500 index could climb moderately.   
 3        

In [23]:
predictions_df = DataProcessing.concat_dfs(batched_predictions_df)
predictions_df

,Base Sentence,Sentence Label,Domain,Model Name,API Name
0,T1: JPMorgan Chase forecasts that the revenue at Amazon will likely decrease in Q3 of 2027.,1,finance,llama-3.3-70b-versatile,GROQ_CLOUD
1,"T2: On 2024-08-21, Citigroup speculates that the net profit at Microsoft will likely increase.",1,finance,llama-3.3-70b-versatile,GROQ_CLOUD
2,T1: Morgan Stanley forecasts that the stock price at Amazon will likely decrease in 2027 Q2.,1,finance,llama-3.1-8b-instant,GROQ_CLOUD
3,"T2: On August 21, 2024, Goldman Sachs speculates that the operating cash flow at Microsoft will likely increase.",1,finance,llama-3.1-8b-instant,GROQ_CLOUD
4,"T3: Morgan Stanley predicts that on September 15, 2025, the S&P 500 index could climb moderately.",1,finance,llama-3.1-8b-instant,GROQ_CLOUD
5,"T4: According to BlackRock, the expected returns at emerging market equities will likely fall in Q4 2026.",1,finance,llama-3.1-8b-instant,GROQ_CLOUD
6,"T5: In Q1 2028, Wells Fargo expects that U.S. Treasury yields will likely stay stable.",1,finance,llama-3.1-8b-instant,GROQ_CLOUD
7,"T6: Apple stock price will likely rise in 2029 Q3, according to Morgan Stanley.",1,finance,llama-3.1-8b-instant,GROQ_CLOUD
8,Here are two unique financial predictions based on the provided templates and examples:,1,finance,llama3-70b-8192,GROQ_CLOUD
9,T1: JPMorgan Chase forecasts that the net profit at Amazon will likely decrease in Q3 2027.,1,finance,llama3-70b-8192,GROQ_CLOUD


In [ ]:
log_directory = '../data/prediction_logs/'
file_name = 'test.log'

In [ ]:
logger = LogData(log_directory, file_name)
logger

In [ ]:
csv_output_path = os.path.join(log_directory, 'from_dataframe.csv')
logger.dataframe_to_csv(predictions_df, csv_output_path)

In [ ]:
csv_input_path = os.path.join(log_directory, 'from_dataframe.csv')
file_name = 'from_csv.log'
logger.csv_to_log(csv_input_path, file_name)

In [ ]:
csv_output_from_log_path = os.path.join(log_directory, 'from_log.csv')
ignore_patterns = ['INFO', 'DEBUG', 'ERROR', 'WARNING', 'CSV Row: '] # Example patterns to ignore

logger.log_to_csv(file_name, csv_output_from_log_path, ignore_patterns)

In [ ]:
csv_input_from_log_path = os.path.join(log_directory, 'from_dataframe.csv')
df_from_log_csv = logger.csv_to_dataframe(csv_input_from_log_path)

if df_from_log_csv is not None:
    print("\nDataFrame created from log CSV:")
    print(df_from_log_csv)
else:
    print("\nFailed to create DataFrame from log CSV. Check the log file for errors.")

In [ ]:
df_from_log_csv